# Using Truss on AWS 

In this guide, we'll cover how to deploy a Truss docker image to AWS using Elastic Container Registry and Elastic Container Service. 

Before we begin, a quick note on prerequisites. For this example, you'll need to have Docker installed. You'll also need two Python packages.

```
pip install --upgrade sklearn truss
```

## Building a model 

Let's build our model. We'll be using `sklearn` to build a simple classifier on the iris dataset. We'll turn that in-memory model to a Truss and then turn that Truss into a docker image. 

In [ ]:
import truss 
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

# Train your model 
iris = load_iris()
data_x = iris['data']
data_y = iris['target']
rfc_model = RandomForestClassifier()
rfc_model.fit(data_x, data_y)

In [ ]:
# Make scaffold 
scaffold = truss.mk_truss(rfc_model, target_directory='test_rfc_1')
# This will produce a folder `test_rfc_1/` relative to your current directory. 
# Now we'll use Truss to build a Docker image on our local system
scaffold.build_docker_image()

That's about all the code we'll need for our model! You should be able to run 

```
docker images
```

and see the resulting image from your Truss. 

## Installing AWS CLI

To push your Truss docker image to ECS, you'll need to have the AWS CLI. 

```
# To download the CLI zip
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
```

To confirm your installation, run`aws --version`. The output should look something like
```
aws-cli/2.7.13 Python/3.9.11 Linux/5.4.0-1074-azure exe/x86_64.debian.11 prompt/off
```

We'll need to authenticate our AWS CLI with our AWS account. AWS has multiple guides on this that depend on your organization or personal account. The simplest way is to run `aws configure` and provide access tokens and keys that can be found on the AWS dashboard. More information [here](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html). 


## Creating a repository on AWS 

Next, you'll have to create an Elastic Container Registry. This will hold your Docker image. To do so:

1. Navigate to the ECR console [here](https://console.aws.amazon.com/ecr/repositories). 
2. Underneath the __Private__ tab, click on __Create Repository__. 
3. Select your visibility settings and give your repository a unique name. You can choose to turn on image scan or KMS encryption. 
4. Press __Create Repository__

You can also use the AWS CLI to create a repository. To do so, run 
```
aws ecr create-repository --repository-name [REPOSITORY_NAME]
```



## Authenticating Docker client
In order to push to the registry, you'll need to authenticate your Docker client to your registry. To find how to do this, 
1. Click into your ECR registry 
2. At the top right, click on __Push commands__
3. Copy the first command; the command that copies the auth token from AWS and authenticates the Docker client to your specific registry. It should look something like 
```
aws ecr get-login-password --region [YOUR_REGION] | docker login --username AWS --password-stdin [AWS_ACCOUNT_ID].dkr.ecr.[AWS_REGION].amazonaws.com
```
4. You should see the command exit with "Login succeeded"

## Pushing our Docker image
Now that we've authenticated our Docker client with our AWS account, it's time to push our Docker image to ECR. 
1. Tag the local Docker image with the following command. Make sure to fill in the variables without the brackets around them. 
```
`docker tag [YOUR_DOCKER_IMAGE_NAME]:latest [AWS_ACCOUNT_ID].dkr.ecr.[AWS_REGION].amazonaws.com/[YOUR_ECR_NAME]:latest
```
2. Push the image to the ECR repository by doing 
```
docker push AWS_ACCOUNT_ID].dkr.ecr.[AWS_REGION].amazonaws.com/[YOUR_ECR_NAME]:latest
```

This'll take some time as our Docker image is 1.5GB. 

## Create ECS Cluster 
We've pushed our image to an AWS ECR repository. Next, we'll need to actually create the ECS cluster that runs that image. 

1. Navigate to the AWS dashboard and select ECS (Elastic Container Service)
2. Press on __Create Cluster__
3. Select the __EC2 Linux + Networking__ template 
4. Give your clster a name. For this example, we'll be using 1 instance of a `t2-medium` with the default Linux 2 AMI. 
5. Click __Create__ at the bottom of the page. It'll take a couple minutes for your ECS cluster to begin. 

## Create a Task definition 
Let's use our ECR image in ECS. To do so, we'll create a task definition. 
1. Navigate back to the ECS dashboard and on the left tab, select __Task Definitions__
2. Click on __Create new Task Definition__ 
3. Because we used the EC2 Linux template in our ECS cluster, we'll use the EC2 launch type. 
4. Give your task definition a name and allocate some memory for the task (we'll use 1gb). 
5. Select __Add Container__ and set __Container name__ to the name of your ECS instance. 
6. For __Image__, copy the URI of your ECR image. It should look something like
7. For __Port mappings__, we'll want to map the host port 80 to container port 8080 (tcp). 
8. Click __Add__ to add the container and then __Create__ to create the task definition. 

## Running your task 
To run the task you just created, navigate to your task definition. 
1. Select __Run task__ underneath __Actions__ on the task page. 
2. On the "Run Task" page, select EC2 as the launch type. 
3. Under __Cluster__, select the ECS cluster you created. 
4. Scroll to the bottom of the page and click __Run Task__. 

## Making a request
Now that you're task is running, you can make requests to your model! To get the public link for your container, navigate to your task and click into the details and you'll see the external link. 

If you've been following along with the model above, you can use the snippet below to make a sample request. 

```
curl -H 'Content-Type: application/json' -d '{"inputs": [[0,0,0,0]]}' -X POST [CONTAINER_LINK]:80/v1/models/model:predict
```